In [1]:
import yfinance as yf
import pandas as pd
from backtesting import Backtest, Strategy
import TI 

C:\Users\ishan\anaconda3\envs\quantVE\lib\site-packages\backtesting\_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

[*********************100%***********************]  1 of 1 completed


In [10]:
data = yf.download("RBLBANK.NS", start="2018-04-01", end='2021-6-15', interval="1d")

data['MACD'] = TI.MACD(data,sp=26,fp=12,p=9)['MACD']
data['MACD_sig'] = TI.MACD(data,sp=26,fp=12,p=9)['signal']
data['BB_down'] = TI.BollBand(data, 14)['BB_dn']
data['ATR'] = TI.ATR(data,14)['ATR']

data.dropna(axis=0, inplace=True)

data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,MACD,MACD_sig,BB_down,ATR
Date,,,,,,,,,,
2018-05-18,511.100006,513.400024,502.149994,508.500000,500.081146,410019,0.479987,1.786006,504.251368,13.221433
2018-05-21,505.549988,508.899994,493.600006,498.200012,489.951691,869992,-0.804756,1.205525,502.696316,13.178578
2018-05-22,497.750000,505.899994,495.000000,500.950012,492.656158,446437,-1.597066,0.592334,501.696722,13.132030
2018-05-23,498.000000,503.799988,495.600006,499.450012,491.181030,352703,-2.309991,-0.030963,499.615691,12.713732
2018-05-24,503.500000,503.500000,492.299988,495.450012,487.247223,268491,-3.139638,-0.688867,496.356169,12.388777


In [3]:
def toseries(value):
    return pd.Series(value)


class MACDandBB(Strategy):
    def init(self):
        price = self.data['Adj Close']
        macs = self.data['MACD_sig']
        mac1 = self.data['MACD']
        BBdn = self.data['BB_down']
        atr = self.data['ATR']
        self.price1 = self.I(toseries, price) 
        self.ma1 = self.I(toseries, macs)
        self.ma2 = self.I(toseries, mac1)
        self.BBdown = self.I(toseries, BBdn)
        self.ATR = self.I(toseries, atr)

    def next(self):
        if self.ma1[-1]>self.ma2[-1]  and self.ma2[-2]>self.ma1[-2] and self.price1[-1]<=self.BBdown[-1]:
            self.position.close()
            self.buy()
        elif self.price1[-1]<= self.price1[-2] - self.ATR[-1]:
            self.position.close()

In [4]:
bt = Backtest(data, MACDandBB, commission=.002,
              exclusive_orders=True, cash=10000)

stats = bt.run()
stats

Start                     2018-05-18 00:00:00
End                       2021-06-14 00:00:00
Duration                   1123 days 00:00:00
Exposure Time [%]                     35.6291
Equity Final [$]                      17751.8
Equity Peak [$]                         20443
Return [%]                            77.5179
Buy & Hold Return [%]                -57.4041
Return (Ann.) [%]                      21.113
Volatility (Ann.) [%]                 29.2851
Sharpe Ratio                         0.720946
Sortino Ratio                         1.38842
Calmar Ratio                         0.787962
Max. Drawdown [%]                    -26.7945
Avg. Drawdown [%]                    -5.13946
Max. Drawdown Duration      344 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                                    8
Win Rate [%]                             62.5
Best Trade [%]                        34.2051
Worst Trade [%]                      -6.66525
Avg. Trade [%]                    

In [8]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,18,42,87,554.607000,560.200012,100.674220,0.010085,2018-07-17,2018-09-24,69 days
1,21,106,131,465.930000,545.000000,1660.470000,0.169704,2018-10-23,2018-11-29,37 days
2,20,183,278,565.629000,628.950012,1266.420244,0.111948,2019-02-12,2019-07-08,146 days
3,21,284,285,603.204000,583.000000,-424.284000,-0.033494,2019-07-16,2019-07-17,1 days
4,41,336,379,300.600000,339.149994,1580.549750,0.128243,2019-10-03,2019-12-09,67 days
5,84,593,637,168.436206,226.050003,4839.558943,0.342051,2020-10-16,2020-12-21,66 days
6,79,681,685,239.077206,239.000000,-6.099283,-0.000323,2021-02-23,2021-03-01,6 days
7,99,713,717,191.782794,179.000000,-1265.496595,-0.066652,2021-04-13,2021-04-20,7 days


In [9]:
bt.plot()